In [43]:
import pandas as pd

In [44]:
chf_patients = pd.read_csv('patients_with_CHF.csv')
chf_patients

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000690,23280645,1,4280,9
1,10000690,25860671,2,42833,9
2,10000690,26146595,3,42832,9
3,10000690,26504700,1,4280,9
4,10000980,20897796,1,I130,10
...,...,...,...,...,...
84710,19998330,24492004,2,42833,9
84711,19998330,27282608,2,42832,9
84712,19998497,24819607,9,4280,9
84713,19998591,23299044,11,4280,9


### total ventilation

In [45]:
mv_patients = pd.read_csv('/media/data/huyennm/mimic-iv/mimic-derived/derived_database/ventilation_durations.csv')
mv_patients

,stay_id,ventnum,starttime,endtime,duration_hours
0,30000153,1,2174-09-29 12:00:00,2174-09-29 17:43:00,5.716667
1,30000213,1,2162-06-21 05:00:00,2162-06-22 07:00:00,26.000000
2,30000831,1,2140-04-18 00:00:00,2140-04-18 08:00:00,8.000000
3,30001148,1,2156-08-30 14:00:00,2156-08-30 17:00:00,3.000000
4,30001656,1,2177-03-15 17:32:00,2177-03-16 10:13:00,16.683333
...,...,...,...,...,...
52537,39997710,8,2134-03-07 20:00:00,2134-03-08 00:00:00,4.000000
52538,39997753,1,2110-09-11 23:00:00,2110-09-12 15:54:00,16.900000
52539,39998012,1,2133-02-03 15:00:00,2133-02-08 18:57:00,123.950000
52540,39998622,1,2135-02-11 18:15:00,2135-02-19 11:35:00,185.333333


In [46]:
icustays = pd.read_csv('/media/data3/biodataset/MIMIC_IV/MIMIC-IV-v3.0/physionet.org/files/mimiciv/3.0/icu/icustays.csv')
icustays.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000690,25860671,37081114,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252
2,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
3,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
4,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113


In [47]:
import time
import sqlite3

In [48]:
print('icustays')
print(icustays.dtypes)
print('mv_chart')
print(mv_patients.dtypes)

icustays
subject_id          int64
hadm_id             int64
stay_id             int64
first_careunit     object
last_careunit      object
intime             object
outtime            object
los               float64
dtype: object
mv_chart
stay_id             int64
ventnum             int64
starttime          object
endtime            object
duration_hours    float64
dtype: object


### get patients who mechanically ventilated on the first day of their ICU stay

In [49]:
ventfirstday_query = """
select
  ie.subject_id, ie.hadm_id, ie.stay_id, ie.intime, ie.outtime, vd.starttime
  -- if vd.stay_id is not null, then they have a valid ventilation event
  -- in this case, we say they are ventilated
  -- otherwise, they are not
  , max(case
      when vd.stay_id is not null then 1
    else 0 end) as vent
from icustays ie
left join mv_patients vd
  on ie.stay_id = vd.stay_id
  and
  (
    -- ventilation duration overlaps with ICU admission -> vented on admission
    (vd.starttime <= ie.intime and vd.endtime >= ie.intime)
    -- ventilation started during the first day
    OR (vd.starttime >= ie.intime and vd.starttime <= DATETIME(ie.intime, '+1 day'))
  )
group by ie.subject_id, ie.hadm_id, ie.stay_id
order by ie.subject_id, ie.hadm_id, ie.stay_id;
"""

In [50]:
conn = sqlite3.connect(":memory:")
chf_patients.to_sql('chf_patients', conn, index=False, if_exists='replace')
mv_patients.to_sql('mv_patients', conn, index=False, if_exists='replace')
icustays.to_sql('icustays', conn, index=False, if_exists='replace')

94458

In [51]:
start_time = time.time()
filtered_df = pd.read_sql_query(ventfirstday_query, conn)
end_time = time.time()
execution_time = end_time - start_time
print(filtered_df)
print(f"Query execution time: {execution_time} seconds")

       subject_id   hadm_id   stay_id               intime  \
0        10000032  29079034  39553978  2180-07-23 14:00:00   
1        10000690  25860671  37081114  2150-11-02 19:37:00   
2        10000980  26913865  39765666  2189-06-27 08:42:00   
3        10001217  24597018  37067082  2157-11-20 19:18:02   
4        10001217  27703517  34592300  2157-12-19 15:42:24   
...           ...       ...       ...                  ...   
94453    19999442  26785317  32336619  2148-11-19 14:23:43   
94454    19999625  25304202  31070865  2139-10-10 19:18:00   
94455    19999828  25744818  36075953  2149-01-08 18:12:00   
94456    19999840  21033226  38978960  2164-09-12 09:26:28   
94457    19999987  23865745  36195440  2145-11-02 22:59:00   

                   outtime            starttime  vent  
0      2180-07-23 23:50:47                 None     0  
1      2150-11-06 17:03:17                 None     0  
2      2189-06-27 20:38:27                 None     0  
3      2157-11-21 22:08:00     

In [52]:
filtered_df.to_csv('vent_firstday.csv', index=False)

In [53]:
filtered_df.to_sql('filtered_df', conn, index=False, if_exists='replace')

94458

### get patients who have CHF and mechanically ventilated on the first day of their ICU stay

In [54]:
query = """
SELECT
    mv.*
FROM chf_patients chf
INNER JOIN filtered_df mv ON chf.subject_id = mv.subject_id AND chf.hadm_id = mv.hadm_id
"""


In [55]:
start_time = time.time()
final_filtered_df = pd.read_sql_query(query, conn)
end_time = time.time()
execution_time = end_time - start_time
print(final_filtered_df)
print(f"Query execution time: {execution_time} seconds")

       subject_id   hadm_id   stay_id               intime  \
0        10000690  25860671  37081114  2150-11-02 19:37:00   
1        10000980  26913865  39765666  2189-06-27 08:42:00   
2        10001843  26133978  39698942  2134-12-05 18:50:03   
3        10002013  23581541  39060235  2160-05-18 10:00:53   
4        10002155  23822395  33685454  2129-08-04 12:45:00   
...           ...       ...       ...                  ...   
27149    19998330  24096838  31199714  2178-11-29 21:51:19   
27150    19998330  24096838  33428243  2178-11-27 22:53:00   
27151    19998330  24492004  32641669  2178-10-01 08:51:00   
27152    19998591  24349193  31144045  2185-07-16 18:48:18   
27153    19998591  24349193  36794489  2185-07-03 22:45:00   

                   outtime            starttime  vent  
0      2150-11-06 17:03:17                 None     0  
1      2189-06-27 20:38:27                 None     0  
2      2134-12-06 14:38:26  2134-12-05 20:00:00     1  
3      2160-05-19 17:33:33  216

In [56]:
final_filtered_df = final_filtered_df[final_filtered_df['vent'] == 1]
final_filtered_df

,subject_id,hadm_id,stay_id,intime,outtime,starttime,vent
2,10001843,26133978,39698942,2134-12-05 18:50:03,2134-12-06 14:38:26,2134-12-05 20:00:00,1
3,10002013,23581541,39060235,2160-05-18 10:00:53,2160-05-19 17:33:33,2160-05-18 14:00:00,1
6,10002428,23473524,35479615,2156-05-11 14:49:34,2156-05-22 14:16:46,2156-05-11 16:00:00,1
9,10002760,28094813,31831386,2141-04-20 13:20:46,2141-04-21 14:26:49,2141-04-20 14:36:00,1
10,10003400,20214994,32128372,2137-02-25 23:37:19,2137-03-10 21:29:36,2137-02-25 23:30:00,1
...,...,...,...,...,...,...,...
27142,19997752,29452285,34531437,2128-03-02 11:09:09,2128-03-07 11:46:51,2128-03-02 13:00:00,1
27144,19997760,25252564,33057362,2190-09-09 22:39:00,2190-10-10 13:57:46,2190-09-09 23:00:00,1
27146,19998330,21135114,31417783,2178-10-21 17:20:00,2178-10-23 17:43:28,2178-10-21 17:00:00,1
27150,19998330,24096838,33428243,2178-11-27 22:53:00,2178-11-29 21:29:39,2178-11-27 22:00:00,1


In [57]:
patients_df = pd.read_csv('/media/data3/biodataset/MIMIC_IV/MIMIC-IV-v3.0/physionet.org/files/mimiciv/3.0/hosp/patients.csv')
patients_df.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000058,F,33,2168,2020 - 2022,NaN
3,10000068,F,19,2160,2008 - 2010,NaN
4,10000084,M,72,2160,2017 - 2019,2161-02-13


In [58]:
final_filtered_df.to_sql('final_filtered_df', conn, index=False, if_exists='replace')
patients_df.to_sql('patients_df', conn, index=False, if_exists='replace')

364627

In [59]:
age_query = """
SELECT
    mv.*,
    pa.anchor_age + (strftime('%Y', mv.starttime) - pa.anchor_year) AS age_at_admission
FROM final_filtered_df mv
INNER JOIN patients_df pa ON mv.subject_id = pa.subject_id
WHERE (pa.anchor_age + (strftime('%Y', mv.starttime) - pa.anchor_year)) > 18
"""

In [60]:
start_time = time.time()
filter_age_df_2 = pd.read_sql_query(age_query, conn)
end_time = time.time()
execution_time = end_time - start_time
print(filter_age_df_2)
print(f"Query execution time: {execution_time} seconds")

       subject_id   hadm_id   stay_id               intime  \
0        10001843  26133978  39698942  2134-12-05 18:50:03   
1        10002013  23581541  39060235  2160-05-18 10:00:53   
2        10002428  23473524  35479615  2156-05-11 14:49:34   
3        10002760  28094813  31831386  2141-04-20 13:20:46   
4        10003400  20214994  32128372  2137-02-25 23:37:19   
...           ...       ...       ...                  ...   
10206    19997752  29452285  34531437  2128-03-02 11:09:09   
10207    19997760  25252564  33057362  2190-09-09 22:39:00   
10208    19998330  21135114  31417783  2178-10-21 17:20:00   
10209    19998330  24096838  33428243  2178-11-27 22:53:00   
10210    19998330  24492004  32641669  2178-10-01 08:51:00   

                   outtime            starttime  vent  age_at_admission  
0      2134-12-06 14:38:26  2134-12-05 20:00:00     1                76  
1      2160-05-19 17:33:33  2160-05-18 14:00:00     1                57  
2      2156-05-22 14:16:46  2156-

In [61]:
filter_age_df_2.drop_duplicates(subset='subject_id')

,subject_id,hadm_id,stay_id,intime,outtime,starttime,vent,age_at_admission
0,10001843,26133978,39698942,2134-12-05 18:50:03,2134-12-06 14:38:26,2134-12-05 20:00:00,1,76
1,10002013,23581541,39060235,2160-05-18 10:00:53,2160-05-19 17:33:33,2160-05-18 14:00:00,1,57
2,10002428,23473524,35479615,2156-05-11 14:49:34,2156-05-22 14:16:46,2156-05-11 16:00:00,1,81
3,10002760,28094813,31831386,2141-04-20 13:20:46,2141-04-21 14:26:49,2141-04-20 14:36:00,1,56
4,10003400,20214994,32128372,2137-02-25 23:37:19,2137-03-10 21:29:36,2137-02-25 23:30:00,1,75
...,...,...,...,...,...,...,...,...
10202,19997293,20208898,31629173,2123-10-12 21:55:18,2123-10-18 02:05:38,2123-10-12 22:00:00,1,76
10205,19997448,23560173,33380030,2122-07-23 10:35:21,2122-07-25 16:40:03,2122-07-23 14:00:00,1,53
10206,19997752,29452285,34531437,2128-03-02 11:09:09,2128-03-07 11:46:51,2128-03-02 13:00:00,1,66
10207,19997760,25252564,33057362,2190-09-09 22:39:00,2190-10-10 13:57:46,2190-09-09 23:00:00,1,53


In [62]:
filter_age_df_2

,subject_id,hadm_id,stay_id,intime,outtime,starttime,vent,age_at_admission
0,10001843,26133978,39698942,2134-12-05 18:50:03,2134-12-06 14:38:26,2134-12-05 20:00:00,1,76
1,10002013,23581541,39060235,2160-05-18 10:00:53,2160-05-19 17:33:33,2160-05-18 14:00:00,1,57
2,10002428,23473524,35479615,2156-05-11 14:49:34,2156-05-22 14:16:46,2156-05-11 16:00:00,1,81
3,10002760,28094813,31831386,2141-04-20 13:20:46,2141-04-21 14:26:49,2141-04-20 14:36:00,1,56
4,10003400,20214994,32128372,2137-02-25 23:37:19,2137-03-10 21:29:36,2137-02-25 23:30:00,1,75
...,...,...,...,...,...,...,...,...
10206,19997752,29452285,34531437,2128-03-02 11:09:09,2128-03-07 11:46:51,2128-03-02 13:00:00,1,66
10207,19997760,25252564,33057362,2190-09-09 22:39:00,2190-10-10 13:57:46,2190-09-09 23:00:00,1,53
10208,19998330,21135114,31417783,2178-10-21 17:20:00,2178-10-23 17:43:28,2178-10-21 17:00:00,1,72
10209,19998330,24096838,33428243,2178-11-27 22:53:00,2178-11-29 21:29:39,2178-11-27 22:00:00,1,72


In [63]:
filter_age_df_2[filter_age_df_2['stay_id'] == 39698942]

,subject_id,hadm_id,stay_id,intime,outtime,starttime,vent,age_at_admission
0,10001843,26133978,39698942,2134-12-05 18:50:03,2134-12-06 14:38:26,2134-12-05 20:00:00,1,76


### having 10210 icustays of 8520 patients which is suitable for the topic

In [64]:
conn.close()

In [65]:
last_charttime_df_2 = filter_age_df_2.loc[filter_age_df_2.groupby('stay_id')['starttime'].idxmax()]
last_charttime_df_2 = last_charttime_df_2.reset_index(drop=True)
last_charttime_df_2

,subject_id,hadm_id,stay_id,intime,outtime,starttime,vent,age_at_admission
0,13180007,27543152,30000213,2162-06-21 05:38:00,2162-06-22 20:52:48,2162-06-21 05:00:00,1,66
1,15726459,22744101,30000831,2140-04-17 21:26:33,2140-04-20 14:21:57,2140-04-18 00:00:00,1,78
2,14311522,24622512,30002548,2111-08-17 13:13:43,2111-08-18 18:50:31,2111-08-17 17:00:00,1,70
3,18730522,21463350,30004391,2153-09-05 13:12:00,2153-09-13 18:21:18,2153-09-05 13:25:00,1,89
4,12509799,25897223,30004530,2165-07-31 09:40:35,2165-08-03 16:29:09,2165-07-31 13:00:00,1,63
...,...,...,...,...,...,...,...,...
10206,19774838,28109956,39992578,2187-09-14 17:39:09,2187-09-16 19:17:27,2187-09-14 21:00:00,1,69
10207,13625156,27789099,39995452,2160-02-07 23:19:01,2160-02-21 16:00:01,2160-02-07 23:30:00,1,71
10208,11539827,21192405,39995735,2124-08-15 10:59:52,2124-08-22 17:01:54,2124-08-15 18:00:00,1,62
10209,12616907,21185436,39996123,2134-07-22 01:52:00,2134-07-25 15:44:52,2134-07-22 12:05:00,1,91


In [68]:
last_charttime_df_2[last_charttime_df_2['stay_id'] == 32456008]

,subject_id,hadm_id,stay_id,intime,outtime,starttime,vent,age_at_admission
2497,15831913,27112445,32456008,2139-06-07 01:20:05,2139-06-11 17:11:42,2139-06-05 22:00:00,1,73


In [67]:
last_charttime_df_2.to_csv('eligible_patients.csv')

In [69]:
last_charttime_df_2.dtypes

subject_id           int64
hadm_id              int64
stay_id              int64
intime              object
outtime             object
starttime           object
vent                 int64
age_at_admission     int64
dtype: object